In [ ]:
import os
import copy
import json
import logging
from glob import glob
from pathlib import Path

from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import widgets, interact, Layout

from flow_control.servoing.playback_env_servo import PlaybackEnvServo

root_dir = "/tmp/flow_experiments3"
task = "shape_sorting"
object_selected = "trapeze" 
task_variant = "rP"  # rotation plus (+-pi)

def get_configurations(root_dir=root_dir, num_episodes=20, prefix=""):
    os.makedirs(root_dir, exist_ok=True)
    save_dir_template = os.path.join(root_dir, f"{prefix}_{task}_{object_selected}")
    for seed in range(num_episodes):
        save_dir = save_dir_template + f"_{task_variant}"+f"_seed{seed:03d}"
        yield object_selected, seed, save_dir
            
demo_cfgs = get_configurations(prefix="demo")
recordings = []
for _, demo_seed, demo_dir in demo_cfgs:
    recordings.append(demo_dir)
    
print("Number of recordings:", len(recordings))
print("first", recordings[0])
print("last ", recordings[-1])

In [ ]:
live_seed, demo_seed = 6, 7
demo_dir = recordings[demo_seed]
print(f"live: {live_seed} demo: {demo_seed} @ {demo_dir}")
print()

demo = PlaybackEnvServo(demo_dir, load="keep")
print("demo keep:", list(demo.keep_dict.keys()))
print()

demo_parts_fn = os.path.join(root_dir, "demo_parts_manual3.json")
with open(demo_parts_fn) as f_obj:
    demo_parts = json.load(f_obj)

demo_keep = sorted(list(demo.keep_dict.keys()))
keep_all = copy.copy(demo.keep_dict)
keep_parts = {}
for p in demo_parts[str(demo_seed)]:
    if p["start"] == 0:
        p_start = -1
    else:
        p_start = p["start"]
        
    parts = []
    for demo_index in demo_keep:
        if p_start < demo_index and p["end"] >= demo_index:
            parts.append(demo_index)
    print(p["name"], '\t', parts)
    
    keep_parts[p["name"]] = dict([(i, demo.keep_dict[i]) for i in parts])
    
# set keep_dict to first part
demo.keep_dict = keep_parts["locate"]
#servo_module


In [ ]:
from math import pi
from flow_control.servoing.module import ServoingModule
from gym_grasping.envs.robot_sim_env import RobotSimEnv
from flow_control.runner import evaluate_control

renderer = "debug"
control_config = dict(mode="pointcloud-abs-rotz", threshold=0.40)

demo = PlaybackEnvServo(recordings[demo_seed])
servo_module = ServoingModule(demo, control_config=control_config,
                              start_paused=False, plot=False, plot_save_dir=None)

env = RobotSimEnv(task='shape_sorting', renderer=renderer, act_type='continuous',
                  initial_pose='close', max_steps=500, control='absolute-full',
                  img_size=(256, 256), param_randomize=("geom",), seed=int(live_seed),
                  task_info=dict(object_rot_range={"rP":pi/2.,"rR":pi/6.}[task_variant]))

_, reward, _, info = evaluate_control(env, servo_module, max_steps=130)

In [ ]:
del(env)